In [1]:
import numpy as np
import netCDF4 as NC
import matplotlib.pyplot as plt
import glob
import sisl
import hubbard

In [2]:
def readNC(fn,ispin=0,nbands=20):
    nc = NC.Dataset(fn)
    kx = nc.variables['grid'][:,0]
    ev = nc.variables['eigenvalues'][:,ispin,-nbands:]
    nc.close()
    return kx,ev

def plot(fn):
    plt.plot()

In [14]:
def TBbands(geom):
    H = hubbard.Hubbard(geom,nsc=[3,1,1],kmesh=[10,1,1])
    H.U = 0
    H.read()
    print H.iterate()
    klist = np.linspace(0,0.5,101)
    eigs = np.empty([len(klist), H.H0.no])
    for ik, k in enumerate(klist):
        eigs[ik,:] = H.Hup.eigh([k,0,0], eigvals_only=True)
    return klist,eigs

In [7]:
def mkplots(t1=2.7,t2=0.2,t3=0.18):
    if t2 > 0 or t3 > 0:
        tb = '3NN'
    else:
        tb = '1NN'

    # Read structure from here:
    #ka,evtb = TBbands('LDA/31-ribbon/8.870/molecule.XV',t1,t2,t3)
    ka,evtb = TBbands('31cGNR.XV')
    ka = 2*ka # Units ka/pi
    egap = (evtb[0,14]+evtb[0,13])/2 # determine midgap

    for xc in [r'LDA', r'GGA']:
        if 'LDA' in xc:
            fn = xc+'/31-ribbon/8.870/Ebands/Electrons.nc' # LDA minimum
        else:
            fn = xc+'/31-ribbon/8.970/Ebands/Electrons.nc' # GGA minimum
        fig = plt.figure(figsize=(4,6));
        axes = plt.axes();

        # Add siesta bands
        k1,ev = readNC(fn)
        egap2 = (ev[0,9]+ev[0,10])/2
        plt.plot(ka,ev-egap2,'k');
        # Add TB results
        plt.plot(ka, evtb-egap, 'r')
        plt.ylim(-4,4)
        plt.rc('font', family='Bitstream Vera Serif', size=16)
        plt.rc('text', usetex=True)

        axes.set_title(xc+r' vs.~'+tb)
        axes.set_xlabel(r'$ka/\pi$')
        axes.set_ylabel(r'$E_{nk}$ (eV)')
        #plt.xticks(np.arange(0,.6,0.1))
        plt.subplots_adjust(left=0.2)
        plt.savefig('test'+xc+'-'+tb+'-%.2f-%.2f-%.2f.pdf'%(t1,t2,t3))
        plt.close('all')


In [16]:
mkplots()

Found 28 pz sites
   U   = 0.0
   Nup = 14
   Ndn = 14
Appending to 31cGNR.XV.nc
(0.0, -109.38174415778576)


In [118]:
for t in [2.7,2.8,2.9]:
    mkplots(t1=t, t2=0.0, t3=0.00)
    mkplots(t1=t, t2=0.2, t3=0.18)

In [117]:
mkplots(t1=2.9, t2=0.0, t3=0.0)

In [84]:
mkplots(t1=2.9, t2=0.2, t3=0.18)